Importo las librerias que voy a utilizar

In [1]:
import numpy as np
import pandas as pd
import time
import random
import math
import copy

Importo los datos del problema

In [2]:
#Matriz de distancias
df_distancias = pd.read_csv("../Datos/Caso II - Matriz de Distancias.csv", delimiter= ",", index_col= 0)

#Informacion de pedidos
datos_prueba = pd.read_csv("../Datos/Caso II - Set de Datos.csv", delimiter= ",", index_col= 0)
column_mapping = {column_name: int(column_name) for column_name in datos_prueba.columns}
datos_prueba.rename(columns=column_mapping, inplace=True) # Reassign columns with new integer names

Defino una serie de constantes para el problema

In [3]:
TARIFA_PROMEDIO = 1200
TARIFA_ESTRATO1 = 1400
TARIFA_ESTRATO2 = 1200
TARIFA_ESTRATO3 = 1000
AFORO_MINIMO = 5600
OCIOSIDAD = 5000
COSTO_DE_OPORTUNIDAD = 3000
AHORRO = 0.05
TARIFA_OBJETIVO = TARIFA_PROMEDIO * (1-AHORRO)
CANTIDAD_DE_CLIENTES_POR_CAMION = 3
CANTIDAD_DE_CAMIONES = 6
TONS_MAX_POR_CAMION = 12
DIST_MAX_POR_CLIENTE = 2

Defino una matriz "Llega" que contiene valor TRUE si la distancia entre dos clientes es <= 2 km y FALSE si la distancia entre dos clientes es > 2 km 

In [4]:
def llega(x):
   return True if x <= DIST_MAX_POR_CLIENTE else False
df_llega = df_distancias.applymap(llega)

Defino la funcion de costo

In [5]:
def costoCamion(tons_repartidas_por_camion):
    costo_por_camion = []
    for tons in tons_repartidas_por_camion:
        costo = float(0)
        if (tons == 0):
            costo = OCIOSIDAD
        elif (tons < 4):
            costo = AFORO_MINIMO
        elif (tons < 6.5):
            costo = tons * TARIFA_ESTRATO1
        elif (tons < 9.5):
            costo = tons * TARIFA_ESTRATO2
        else:
            costo = tons * TARIFA_ESTRATO3
        costo_por_camion.append(costo)
    return costo_por_camion

Defino la funcion que genera la solucion inicial

In [6]:
#Funcion que genera la solucion inicial
    #Input:     pedidos - pandas series con los pedidos del dia
    #Output:    clientes_por_camion = [['L'], ['J', 'D'], ['K'], [], ['G', 'E'], ['I', 'H', 'B']]
    #           toneladas_por_camion = [6, 9, 1, 0, 7, 16]
    #           toneladas_repartidas_por_camion = [12, 4, 5, 10, 0, 3]
    #           costo_toneladas_repartidas_por_camion = [12000, 5600, 7000, 10000, 5000, 5600]
    #           pedidos_no_asignados = ['D', 'C', 'A']
    #           toneladas_no_asignadas = 13
    #           toneladas_no_repartidas = 0
    #           costo_total = 84200

def generarSolucionInicial(p):

    #Elimina los clientes con 0 ton, toma el index (nombre de los clientes), lo convierto en una lista y elimino el ultimo que es el total
    pedidos = p[p > 0].index
    pedidos = pedidos.to_list()
    pedidos.pop()

    #Inicializo las siguientes variables
    pedidos_por_asignar = pedidos                                   #Inicializo una lista con los pedidos a asignar
    clientes_por_camion = [[] for _ in range(CANTIDAD_DE_CAMIONES)] #Inicializo una lista con los pedidos asignados a cada camion
    clientes_no_asignados = []                                      #Inicializo un lista con los pedidos no asignados
    
    #Defino la cantidad de clientes por asignar
    cantidad_clientes = len(pedidos_por_asignar)

    #Recorro toda la lista de clientes por asignar
    for i in range(cantidad_clientes): 
        cliente = pedidos_por_asignar.pop()                     #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)        #Genero un numero aleatorio entero del 0 al 5 para elegir a que camion asignar dicho pedido
        camion_cliente = clientes_por_camion[rand]              #Defino los clientes asignados hasta el momento a ese camion
        cantidad_clientes = len(camion_cliente)                 #Determino la cantidad de clientes asignados hasta el momento a ese camion
        
        #Verifico que se cumplan todas las restricciones
            #RESTRICCION 1: "Los camiones no pueden salir con más de X entregas por día."
            #RESTRICCION 2: "No se pueden superar los 2 km de distancia entre cliente y cliente de un mismo camion"
        #Primero verifico la restriccion 1, si la cumple verifico la restriccion 2, si la cumple, asigno el cliente a ese camion
        #Si no cumple alguna de las condiciones lo considero un pedido no asignado

        if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
            llega_list = []
            llega_todos = None
            for cliente_i in camion_cliente:
                llega = df_llega.loc[cliente_i,cliente]
                llega_list.append(llega)

            llega_todos = all(llega_list)              #llega_todos = 0 si hay al menos un FALSE y llega_todos = 1 si todos son TRUE
            if(llega_todos):                                    #Verifico que la distancia entre clientes sea menor a la maxima
                clientes_por_camion[rand].append(cliente)
            else:
                clientes_no_asignados.append(cliente)
        else:
            clientes_no_asignados.append(cliente)

    pedidos_por_camion = []                                    
    toneladas = []                                              #Inicializo una lista con las toneladas por camion
    toneladas_no_asignadas = 0                                  #Inicializo una variable con las toneladas no asignadas

    #Recorro los clientes asignados a cada camion
    for camion in clientes_por_camion:                         
        pedido = p[camion]                                      #Defino las toneladas pedidas por cada cliente 
        pedidos_por_camion.append(pedido)

    for pedidos in pedidos_por_camion:
        tons = pedidos.values.sum()
        toneladas.append(tons)
    
    #Calculo la cantidad de toneladas no asignadas
    toneladas_no_asignadas = p[clientes_no_asignados].values.sum()  

    tons_repartidas_por_camion = []                             #Inicializo una lista con las toneladas repartidas por camion
    toneladas_no_repartidas = 0                                 #Inicializo una variable con las toneladas no repartidas

    #De las toneladas asignadas a cada camion, calculo cuantas efectivamente se reparten y cuantas no
    for t in range(0,6):
        tons = toneladas[t] 
        if(tons <= TONS_MAX_POR_CAMION):
            tons_repartidas_por_camion.append(tons)
        else:
            tons_repartidas_por_camion.append(TONS_MAX_POR_CAMION)
            toneladas_no_repartidas += tons-TONS_MAX_POR_CAMION
   
    #Calculo la cantidad de toneladas repartidas
    toneladas_repartidas = sum(tons_repartidas_por_camion)
    
    costo_por_camion = []
    
    #Calculo el costo por camion
    costo_por_camion = costoCamion(tons_repartidas_por_camion)
   
    #Calculo del costo total
    costo_total = sum(costo_por_camion) + (toneladas_no_asignadas + toneladas_no_repartidas)*COSTO_DE_OPORTUNIDAD

    #Calculo el costo por tonelada repartida
    costo_por_ton = costo_total / toneladas_repartidas
  
    return clientes_por_camion, clientes_no_asignados, toneladas, toneladas_no_asignadas, tons_repartidas_por_camion, toneladas_no_repartidas, costo_por_camion, costo_total, toneladas_repartidas, costo_por_ton

Defino la funcion que genera el vecino

In [7]:
#Funcion que genera el vecino
def generarVecino(p, clientes_por_camion, clientes_no_asignados): 
    
    #Elimino un pedido aleatorio por camion para reasignarlo nuevamente
    for i in range(CANTIDAD_DE_CAMIONES):

        clientes_por_camion_i = clientes_por_camion[i]          #Me quedo con los pedidos del camion i
        len_i = len(clientes_por_camion_i)                      #Veo cuantos pedidos tiene el camion
        
        if(len_i > 0):
            rand = random.randint(0, len_i-1)                     #Selecciono aleatoriamente el pedido a cambiar
            cliente_eliminado = clientes_por_camion_i.pop(rand)
            clientes_no_asignados.append(cliente_eliminado)      #Lo agrego a la lista de clientes no asignados
    
    #Asigno los clientes no asignados a los camiones aleatoriamente
    
    #Inicializo las siguientes variables
    pedidos_por_asignar = clientes_no_asignados                     #Inicializo una lista con los pedidos a asignar
    clientes_no_asignados = []                                      #Inicializo un lista con los pedidos no asignados
    
    #Defino la cantidad de clientes por asignar
    cantidad_clientes = len(pedidos_por_asignar)

    #Recorro toda la lista de clientes por asignar
    for i in range(cantidad_clientes): 
        cliente = pedidos_por_asignar.pop()                     #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)        #Genero un numero aleatorio entero del 0 al 5 para elegir a que camion asignar dicho pedido
        camion_cliente = clientes_por_camion[rand]              #Defino los clientes asignados hasta el momento a ese camion
        cantidad_clientes = len(camion_cliente)                 #Determino la cantidad de clientes asignados hasta el momento a ese camion
        
        #Verifico que se cumplan todas las restricciones
            #RESTRICCION 1: "Los camiones no pueden salir con más de X entregas por día."
            #RESTRICCION 2: "No se pueden superar los 2 km de distancia entre cliente y cliente de un mismo camion"
        #Primero verifico la restriccion 1, si la cumple verifico la restriccion 2, si la cumple, asigno el cliente a ese camion
        #Si no cumple alguna de las condiciones lo considero un pedido no asignado

        if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
            llega_list = []
            for cliente_i in camion_cliente:
                llega = df_llega.loc[cliente_i,cliente]
                llega_list.append(llega)
            llega_todos = all(llega_list)       #llega_todos = 0 si hay al menos un FALSE y llega_todos = 1 si todos son TRUE
            if(llega_todos):                                    #Verifico que la distancia entre clientes sea menor a la maxima
                clientes_por_camion[rand].append(cliente)
            else:
                clientes_no_asignados.append(cliente)
        else:
            clientes_no_asignados.append(cliente)

    pedidos_por_camion = []                                    
    toneladas = []                                              #Inicializo una lista con las toneladas por camion
    toneladas_no_asignadas = 0                                  #Inicializo una variable con las toneladas no asignadas

    #Recorro los clientes asignados a cada camion
    for camion in clientes_por_camion:                         
        pedido = p[camion]                                      #Defino las toneladas pedidas por cada cliente 
        pedidos_por_camion.append(pedido)
    for pedidos in pedidos_por_camion:
        tons = pedidos.values.sum()
        toneladas.append(tons)
    
    #Calculo la cantidad de toneladas no asignadas
    toneladas_no_asignadas = p[clientes_no_asignados].values.sum()  

    tons_repartidas_por_camion = []                             #Inicializo una lista con las toneladas repartidas por camion
    toneladas_no_repartidas = 0                                 #Inicializo una variable con las toneladas no repartidas

    #De las toneladas asignadas a cada camion, calculo cuantas efectivamente se reparten y cuantas no
    for t in range(0,6):
        tons = toneladas[t]
        if(tons <= TONS_MAX_POR_CAMION):
            tons_repartidas_por_camion.append(tons)
        else:
            tons_repartidas_por_camion.append(TONS_MAX_POR_CAMION)
            toneladas_no_repartidas += tons-TONS_MAX_POR_CAMION

    #Calculo la cantidad de toneladas repartidas
    toneladas_repartidas = sum(tons_repartidas_por_camion)
    
    costo_por_camion = []
    
    #Calculo el costo por camion
    costo_por_camion = costoCamion(tons_repartidas_por_camion)
   
    #Calculo del costo total
    costo_total = sum(costo_por_camion) + (toneladas_no_asignadas + toneladas_no_repartidas)*COSTO_DE_OPORTUNIDAD

    #Calculo el costo por tonelada repartida
    costo_por_ton = costo_total / toneladas_repartidas
  
    return clientes_por_camion, clientes_no_asignados, toneladas, toneladas_no_asignadas, tons_repartidas_por_camion, toneladas_no_repartidas, costo_por_camion, costo_total, toneladas_repartidas, costo_por_ton   

Parámetros del RECOCIDO SIMULADO

In [8]:
T_INICIAL = 1500
T_FINAL = 10
PASO = -1
ITERACIONES_POR_TEMPERATURA = 50

Defino la funcion que genera el recocido simulado

In [9]:
def recocidoSimulado(dia):

    #Inicializo una serie de variables
    clientes_por_camion_min = []
    clientes_no_asignados_min = []
    costo_por_ton_min = float('inf')

    #Genero la solucion inicial
    x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = generarSolucionInicial(dia)
    #clientes_por_camion, clientes_no_asignados, toneladas, toneladas_no_asignadas, tons_repartidas_por_camion, toneladas_no_repartidas, costo_por_camion, costo_total, toneladas_repartidas, costo_por_ton

    #Guardo la solucion actual
    clientes_por_camion_now = x1
    clientes_no_asignados_now = x2
    costo_por_ton_now = x10

    #Guardo la solucion minima 
    clientes_por_camion_min = x1
    clientes_no_asignados_min = x2
    costo_por_ton_min = x10

    #Comienzo el recocido simulado

    for t in range(T_INICIAL, T_FINAL, PASO):

        #Para cada temperatura realizo varias iteraciones
        for n in range(ITERACIONES_POR_TEMPERATURA):
            
            #Defino un vecino
            y1, y2, y3, y4, y5, y6, y7, y8, y9, y10 = generarVecino(dia, copy.deepcopy(clientes_por_camion_now),copy.deepcopy(clientes_no_asignados_now))
            #clientes_por_camion, clientes_no_asignados, toneladas, toneladas_no_asignadas, tons_repartidas_por_camion, toneladas_no_repartidas, costo_por_camion, costo_total, toneladas_repartidas, costo_por_ton
            
            #Calculo la diferencia entre la solucion actual y la nueva solucion
            delta = costo_por_ton_now - y10
                        
            #¿Es mejor que la solucion actual?

            if delta < 0 :                              #Si la solucion nueva es peor que la anterio delta < 0 porque estoy minimizando el costo
                
                #Como no es mejor, debo aplica un proceso de aceptacion estocastico

                acepto = math.e**(delta/t)              #Calculo la probabilidad de aceptar dicha solucion peor 
                probabilidad = random.random()          #Genero un numero aleatorio
            
                if acepto > probabilidad:               #Si el número generado es menor que la probabilidad de aceptar, entonces ACEPTO UNA PEOR SOLUCION
                    
                    clientes_por_camion_now = y1
                    clientes_no_asignados_now = y2
                    costo_por_ton_now = y10

            else:                                       #Si la solucion nueva es mejor que la anterior la acepto             
                clientes_por_camion_now = y1
                clientes_no_asignados_now = y2
                costo_por_ton_now = y10

            #¿Es mejor que la condicion minima?

            if costo_por_ton_now < costo_por_ton_min:   #Si la solucion actual es mejor que la minima, actualizo las variables

                clientes_por_camion_min = clientes_por_camion_now
                clientes_no_asignados_min = clientes_no_asignados_now
                costo_por_ton_min = costo_por_ton_now

    return clientes_por_camion_min, clientes_no_asignados_min, costo_por_ton_min

Simulo un dia

In [10]:
dia = datos_prueba[1]
inicio = time.time()
clientes_por_camion_resultado, clientes_no_asignados_resultado, costo_por_ton_resultado = recocidoSimulado(dia)
fin = time.time()  
print("Clientes por camion: ", clientes_por_camion_resultado)
print("Clientes no asignados: ", clientes_no_asignados_resultado)
print("Costo por tonelada: ", costo_por_ton_resultado)
print("Tiempo de ejecucion: ", fin - inicio)

Clientes por camion:  [['J', 'E', 'H'], ['L', 'I'], ['D', 'F'], ['A'], ['B', 'C'], ['G', 'K']]
Clientes no asignados:  []
Costo por tonelada:  1097.872340425532
Tiempo de ejecucion:  329.20752906799316


In [11]:
def generarSolucionInicial(p):

    #Elimina los clientes con 0 ton, toma el index (nombre de los clientes), lo convierto en una lista y elimino el ultimo que es el total
    pedidos = p[p > 0].index
    pedidos = pedidos.to_list()
    pedidos.pop()

    #Inicializo las siguientes variables
    pedidos_por_asignar = pedidos  
    #CLIENTES POR CAMION                                 #Inicializo una lista con los pedidos a asignar
    cpc = [[] for _ in range(CANTIDAD_DE_CAMIONES)] #Inicializo una lista con los pedidos asignados a cada camion
    #CLIENTES NO ASIGNADOS
    cna = []                                      #Inicializo un lista con los pedidos no asignados
    
    #Defino la cantidad de clientes por asignar
    cantidad_clientes = len(pedidos_por_asignar)

    #Recorro toda la lista de clientes por asignar
    for i in range(cantidad_clientes): 
        cliente = pedidos_por_asignar.pop()                     #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)        #Genero un numero aleatorio entero del 0 al 5 para elegir a que camion asignar dicho pedido
        camion_cliente = cpc[rand]              #Defino los clientes asignados hasta el momento a ese camion
        cantidad_clientes = len(camion_cliente)                 #Determino la cantidad de clientes asignados hasta el momento a ese camion
        
        #Verifico que se cumplan todas las restricciones
            #RESTRICCION 1: "Los camiones no pueden salir con más de X entregas por día."
            #RESTRICCION 2: "No se pueden superar los 2 km de distancia entre cliente y cliente de un mismo camion"
        #Primero verifico la restriccion 1, si la cumple verifico la restriccion 2, si la cumple, asigno el cliente a ese camion
        #Si no cumple alguna de las condiciones lo considero un pedido no asignado

        if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
            llega_list = []
            llega_todos = None
            for cliente_i in camion_cliente:
                llega = df_llega.loc[cliente_i,cliente]
                llega_list.append(llega)

            llega_todos = all(llega_list)              #llega_todos = 0 si hay al menos un FALSE y llega_todos = 1 si todos son TRUE
            if(llega_todos):                                    #Verifico que la distancia entre clientes sea menor a la maxima
                cpc[rand].append(cliente)
            else:
                cna.append(cliente)
        else:
            cna.append(cliente)
    return cpc, cna

In [12]:
#Recibe clientes_por_camion (cpc), clientes_no_asignados (cna), pedidos(p)
def calcularCosto(cpc, cna, p):
    pedidos_por_camion = []                                    
    toneladas = []                                              #Inicializo una lista con las toneladas por camion
    toneladas_no_asignadas = 0                                  #Inicializo una variable con las toneladas no asignadas

    #Recorro los clientes asignados a cada camion
    for camion in cpc:                         
        pedido = p[camion]                                      #Defino las toneladas pedidas por cada cliente 
        pedidos_por_camion.append(pedido)

    for pedidos in pedidos_por_camion:
        tons = pedidos.values.sum()
        toneladas.append(tons)
    
    #Calculo la cantidad de toneladas no asignadas
    toneladas_no_asignadas = p[cna].values.sum()  

    tons_repartidas_por_camion = []                             #Inicializo una lista con las toneladas repartidas por camion
    toneladas_no_repartidas = 0                                 #Inicializo una variable con las toneladas no repartidas

    #De las toneladas asignadas a cada camion, calculo cuantas efectivamente se reparten y cuantas no
    for tons in toneladas:
        if(tons <= TONS_MAX_POR_CAMION):
            tons_repartidas_por_camion.append(tons)
        else:
            tons_repartidas_por_camion.append(TONS_MAX_POR_CAMION)
            toneladas_no_repartidas += tons-TONS_MAX_POR_CAMION
   
    #Calculo la cantidad de toneladas repartidas
    toneladas_repartidas = sum(tons_repartidas_por_camion)
    
    costo_por_camion = []
    
    #Calculo el costo por camion
    costo_por_camion = costoCamion(tons_repartidas_por_camion)
   
    #Calculo del costo total
    costo_total = sum(costo_por_camion) + (toneladas_no_asignadas + toneladas_no_repartidas)*COSTO_DE_OPORTUNIDAD

    #Calculo el costo por tonelada repartida
    costo_por_ton = costo_total / toneladas_repartidas

    return costo_por_ton

In [13]:
def generarVecino(cpc, cna): 
    
    #Elimino un pedido aleatorio por camion para reasignarlo nuevamente
    for i in range(CANTIDAD_DE_CAMIONES):

        clientes_por_camion_i = cpc[i]          #Me quedo con los pedidos del camion i
        len_i = len(clientes_por_camion_i)                      #Veo cuantos pedidos tiene el camion
        
        if(len_i > 0):
            rand = random.randint(0, len_i-1)                     #Selecciono aleatoriamente el pedido a cambiar
            cliente_eliminado = clientes_por_camion_i.pop(rand)
            cna.append(cliente_eliminado)      #Lo agrego a la lista de clientes no asignados
    
    #Asigno los clientes no asignados a los camiones aleatoriamente
    
    #Inicializo las siguientes variables
    pedidos_por_asignar = cna                     #Inicializo una lista con los pedidos a asignar
    cna = []                                      #Inicializo un lista con los pedidos no asignados
    
    #Defino la cantidad de clientes por asignar
    cantidad_clientes = len(pedidos_por_asignar)

    #Recorro toda la lista de clientes por asignar
    for i in range(cantidad_clientes): 
        cliente = pedidos_por_asignar.pop()                     #Elimino el ultimo cliente de la lista de clientes a la vez que le asigno el cliente eliminado a la variable cliente
        rand = random.randint(0, CANTIDAD_DE_CAMIONES-1)        #Genero un numero aleatorio entero del 0 al 5 para elegir a que camion asignar dicho pedido
        camion_cliente = cpc[rand]              #Defino los clientes asignados hasta el momento a ese camion
        cantidad_clientes = len(camion_cliente)                 #Determino la cantidad de clientes asignados hasta el momento a ese camion
        
        #Verifico que se cumplan todas las restricciones
            #RESTRICCION 1: "Los camiones no pueden salir con más de X entregas por día."
            #RESTRICCION 2: "No se pueden superar los 2 km de distancia entre cliente y cliente de un mismo camion"
        #Primero verifico la restriccion 1, si la cumple verifico la restriccion 2, si la cumple, asigno el cliente a ese camion
        #Si no cumple alguna de las condiciones lo considero un pedido no asignado

        if cantidad_clientes < CANTIDAD_DE_CLIENTES_POR_CAMION: #Verifico que el camion no tenga 3 clientes
            llega_list = []
            for cliente_i in camion_cliente:
                llega = df_llega.loc[cliente_i,cliente]
                llega_list.append(llega)
            llega_todos = all(llega_list)       #llega_todos = 0 si hay al menos un FALSE y llega_todos = 1 si todos son TRUE
            if(llega_todos):                                    #Verifico que la distancia entre clientes sea menor a la maxima
                cpc[rand].append(cliente)
            else:
                cna.append(cliente)
        else:
            cna.append(cliente)
        
        return cpc, cna

In [14]:
cna = ['A','B']

pedidos_por_asignar = cna                     #Inicializo una lista con los pedidos a asignar
cna = []   
print(pedidos_por_asignar) 

['A', 'B']


In [15]:
def opt_2(cpc):
    #ELIJO 2 CAMIONES DIFERENTES AL AZAR
    #genero 2 numeros al azar diferentes
    rand1 = random.randint(0, CANTIDAD_DE_CAMIONES-1)
    while True:
        rand2 = random.randint(0, CANTIDAD_DE_CAMIONES-1)
        if rand1 != rand2:
            break
    #genero los 2 camiones
    camion_1 = cpc[rand1]
    camion_2 = cpc[rand2]

    #SELECCIONO UN CLIENTE AL AZAR DE CADA CAMION
    rand_cliente_1 = random.randint(0, len(camion_1) - 1 )
    rand_cliente_2 = random.randint(0, len(camion_2) - 1 )

    #Elimino el cliente del camion    
    cliente_2 = camion_1.pop(rand_cliente_1)
    cliente_1 = camion_2.pop(rand_cliente_2)

    #Verifico que se puedan poner uno en cada lugar]
    #Pruebo si puedo meter al cliente_1 en el camion_2
    llega_list_1 = []
    for cliente in camion_2:
        llega_1 = df_llega.loc[cliente,cliente_2]
        llega_list_1.append(llega_1)
    llega_todos_1 = all(llega_list_1)
    
    llega_list_2 = []
    for cliente in camion_1:
        llega_2 = df_llega.loc[cliente,cliente_1]
        llega_list_2.append(llega_2)
    llega_todos_2 = all(llega_list_2)

    #Si tanto el cliente_1 y el cliente_2 estan a menos de 2km de los otros clientes del camion los asigno cruzo y si no los dejo como estaban
    if(llega_todos_1 and llega_todos_2):
        #Los asigno cruzados
        camion_1.append(cliente_1)
        camion_2.append(cliente_2)
    else:
        #Los dejo como estaban
        camion_1.append(cliente_2)
        camion_2.append(cliente_1)


    #Retorno los clientes por camion
    return cpc

In [37]:
def recocidoSimulado1(pedidos):
    clientes_por_camion_min = []
    clientes_no_asignados_min = []
    costo_por_ton_min = float('inf')

    clientes_por_camion, clientes_no_asignados = generarSolucionInicial(pedidos)
    costo = calcularCosto(clientes_por_camion, clientes_no_asignados, pedidos)
    
    clientes_por_camion_1 = clientes_por_camion
    clientes_no_asignados_1 = clientes_no_asignados
    costo_1 = costo

    clientes_por_camion_min = clientes_por_camion
    clientes_no_asignados_min = clientes_no_asignados
    costo_min = costo


    for t in range(T_INICIAL, T_FINAL, PASO):
        for n in range(ITERACIONES_POR_TEMPERATURA):
            clientes_por_camion, clientes_no_asignados = generarSolucionInicial(pedidos)
            costo = calcularCosto(clientes_por_camion, clientes_no_asignados,pedidos)

            delta = costo_1 - costo

            if delta < 0:

                p = math.e**(delta/t)
                rand_0_1 = random.random()

                if p > rand_0_1:
                        
                        clientes_por_camion_1 = clientes_por_camion
                        clientes_no_asignados_1 = clientes_no_asignados
                        costo_1 = costo
            else:
                        clientes_por_camion_1 = clientes_por_camion
                        clientes_no_asignados_1 = clientes_no_asignados
                        costo_1 = costo

                        if costo < costo_min:
                            clientes_por_camion_min = clientes_por_camion
                            clientes_no_asignados_min = clientes_no_asignados
                            costo_min = costo

    print(clientes_por_camion_min, clientes_no_asignados_min, costo_min)       
    for i in range(ITERACIONES_2OPT):
        clientes_por_camion = opt_2(clientes_por_camion_min)
        costo = calcularCosto(clientes_por_camion, clientes_no_asignados_min, pedidos)
        if costo < costo_min:
            clientes_por_camion_min = clientes_por_camion
            costo_min = costo

    return clientes_por_camion_min, clientes_no_asignados_min, costo_min





In [17]:
TARIFA_PROMEDIO = 1200
TARIFA_ESTRATO1 = 1400
TARIFA_ESTRATO2 = 1200
TARIFA_ESTRATO3 = 1000
AFORO_MINIMO = 5600
OCIOSIDAD = 5000
COSTO_DE_OPORTUNIDAD = 3000
AHORRO = 0.05
TARIFA_OBJETIVO = TARIFA_PROMEDIO * (1-AHORRO)
CANTIDAD_DE_CLIENTES_POR_CAMION = 3
CANTIDAD_DE_CAMIONES = 6
TONS_MAX_POR_CAMION = 12
DIST_MAX_POR_CLIENTE = 2

In [30]:
CANTIDAD_DE_CAMIONES = 5
T_INICIAL = 1500
T_FINAL = 500
PASO = -1
ITERACIONES_POR_TEMPERATURA = 5
ITERACIONES_2OPT = 1000

In [36]:
CANTIDAD_DE_CAMIONES = 5
T_INICIAL = 1500
T_FINAL = 500
PASO = -1
ITERACIONES_POR_TEMPERATURA = 5
ITERACIONES_2OPT = 1000

for i in range(1,10):
    print(recocidoSimulado1(datos_prueba[i]))

([['G', 'K', 'J'], ['C', 'L'], ['I', 'F'], ['E', 'A'], ['B', 'D', 'H']], [], 1034.0425531914893)
([['L', 'C'], ['D', 'B', 'A'], ['I', 'F', 'J'], ['H', 'K'], ['E', 'G']], [], 1033.3333333333333)
([['E', 'G'], ['J', 'L', 'B'], ['D', 'F'], ['K', 'I', 'H'], ['A', 'C']], [], 1029.090909090909)
([['E', 'G', 'J'], ['I', 'A'], ['C', 'L'], ['B', 'F'], ['K', 'H']], ['D'], 1111.111111111111)
([['L', 'K', 'I'], ['M', 'C', 'D'], ['N', 'J', 'G'], ['A', 'E'], ['F', 'H', 'O']], ['B'], 1355.9322033898304)
([['O', 'E', 'H'], ['G', 'N'], ['I', 'A', 'M'], ['F', 'B', 'J'], ['L', 'D', 'C']], ['K'], 1210.5263157894738)
([['E', 'F', 'O'], ['L', 'I'], ['M', 'N'], ['H', 'J', 'B'], ['C', 'A', 'D']], ['K', 'G'], 1226.4150943396226)
([['L', 'N'], ['M', 'A', 'I'], ['E', 'J', 'B'], ['D', 'O'], ['K', 'G']], ['H', 'F', 'C'], 1650.0)
([['K', 'I'], ['G', 'F', 'E'], ['A', 'O', 'D'], ['H', 'M', 'J'], ['L', 'B', 'N']], ['C'], 1350.0)


In [39]:
CANTIDAD_DE_CAMIONES = 6
T_INICIAL = 3000
T_FINAL = 500
PASO = -1
ITERACIONES_POR_TEMPERATURA = 5
ITERACIONES_2OPT = 1000
for i in range(1,10):
    print(recocidoSimulado1(datos_prueba[i]))


[['J', 'G'], ['E'], ['I', 'C', 'B'], ['F', 'D'], ['L', 'K', 'H'], ['A']] [] 1102.127659574468
([['F', 'G'], ['K'], ['A', 'L', 'I'], ['H', 'J'], ['E', 'D', 'C'], ['B']], [], 1102.127659574468)
[['K'], ['L', 'I', 'B'], ['H', 'G'], ['F', 'C', 'A'], ['J'], ['E', 'D']] [] 1081.4814814814815
([['A'], ['F', 'E', 'H'], ['L', 'K'], ['J', 'I', 'D'], ['G'], ['C', 'B']], [], 1081.4814814814815)
[['F', 'D'], ['G'], ['C', 'A'], ['L', 'J'], ['K', 'H'], ['I', 'E', 'B']] [] 1080.0
([['L', 'K'], ['B'], ['C', 'I'], ['D', 'H'], ['J', 'G'], ['F', 'E', 'A']], [], 1072.7272727272727)
[['G'], ['K', 'H'], ['E', 'A'], ['F', 'C'], ['L', 'D', 'B'], ['J', 'I']] [] 1067.857142857143
([['F'], ['G', 'H'], ['B', 'J'], ['D', 'A'], ['C', 'I', 'E'], ['K', 'L']], [], 1053.5714285714287)
[['J', 'I', 'G'], ['M', 'E', 'B'], ['F', 'D'], ['C', 'A'], ['O', 'H'], ['N', 'L', 'K']] [] 1115.625
([['E', 'N', 'F'], ['G', 'I', 'K'], ['A', 'C'], ['H', 'L'], ['J', 'O'], ['B', 'M', 'D']], [], 1000.0)
[['L', 'A'], ['K', 'H'], ['O', 'N', '

In [40]:
CANTIDAD_DE_CAMIONES = 6
T_INICIAL = 3000
T_FINAL = 500
PASO = -1
ITERACIONES_POR_TEMPERATURA = 5
ITERACIONES_2OPT = 1000
for i in range(1,10):
    print('---------------------------Dia', i,'---------------------------')
    print(recocidoSimulado1(datos_prueba[i]))



---------------------------Dia 1 ---------------------------
[['K', 'G'], ['F', 'D', 'C'], ['E'], ['A'], ['L', 'J', 'I'], ['H', 'B']] [] 1102.127659574468
([['B', 'L'], ['E', 'D', 'F'], ['C'], ['A'], ['J', 'H', 'K'], ['I', 'G']], [], 1102.127659574468)
---------------------------Dia 2 ---------------------------
[['J', 'I'], ['D', 'C'], ['F', 'E', 'A'], ['H', 'G'], ['K'], ['L', 'B']] [] 1062.962962962963
([['C', 'A'], ['L', 'J'], ['K', 'H', 'G'], ['D', 'B'], ['I'], ['E', 'F']], [], 1051.851851851852)
---------------------------Dia 3 ---------------------------
[['C', 'B'], ['I', 'H'], ['G', 'F', 'E'], ['K', 'J'], ['L'], ['D', 'A']] [] 1076.3636363636363
([['J', 'I'], ['G', 'K'], ['E', 'C', 'D'], ['L', 'H'], ['F'], ['B', 'A']], [], 1072.7272727272727)
---------------------------Dia 4 ---------------------------
[['L', 'B', 'A'], ['G'], ['J', 'E'], ['K', 'I'], ['H', 'F'], ['D', 'C']] [] 1053.5714285714287
([['A', 'C', 'L'], ['K'], ['B', 'D'], ['F', 'I'], ['G', 'H'], ['E', 'J']], [], 1050